In [101]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
path = 'gdrive/MyDrive/MadeProject'

Mounted at /content/gdrive/


In [2]:
# !git clone https://github.com/NVIDIA/DeepLearningExamples
!git clone https://github.com/GimmeDanger/made-rec-sys-2021.git
ncf_path = '/content/made-rec-sys-2021/ncf'

Cloning into 'made-rec-sys-2021'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 145 (delta 77), reused 99 (delta 36), pack-reused 0
Receiving objects: 100% (145/145), 301.98 KiB | 2.65 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [ ]:
# intall requirements: apex + ncf
!cd '{ncf_path}' && python3 -m pip install -r requirements.txt

# https://stackoverflow.com/questions/66610378/unencryptedcookiesessionfactoryconfig-error-when-importing-apex
!git clone https://github.com/NVIDIA/apex
!cd apex && python3 -m pip install -v --disable-pip-version-check --no-cache-dir \
--global-option="--cpp_ext" --global-option="--cuda_ext" ./

In [102]:
import pandas as pd

!mkdir -p /data
!mkdir -p /data/cache/interactions

!cp gdrive/MyDrive/MadeProject/ncf_orders.parquet /data/interactions.parquet
!cp gdrive/MyDrive/MadeProject/ncf_val_df_part_1.parquet /data/pred_interactions.parquet

In [103]:
!cd '{ncf_path}' && python3 convert.py \
    --train_path /data/interactions.parquet \
    --pred_path /data/pred_interactions.parquet \
    --output /data/cache/interactions \
    --valid_negative=100

Loading raw data from /data/interactions.parquet
Mapping original user and item IDs to new sequential IDs
535820 598092
Generating validation negatives...
100% 59809200/59809200 [00:35<00:00, 1674383.12it/s]
Loading pred data from /data/pred_interactions.parquet
pred len, uniq users, uniq items: 30000000 39378 7152
filtering by user_to_index
pred len, uniq users, uniq items: 30000000 39378 7152
filtering by item_to_index
pred len, uniq users, uniq items: 30000000 39378 7152


In [104]:
!cd '{ncf_path}' && python3 -m torch.distributed.launch --nproc_per_node=1 \
    --use_env ncf.py --data /data/cache/interactions \
    --checkpoint_dir /data/checkpoints/ \
    -k 30

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
DLL 2021-12-08 00:21:58.752588 - PARAMETER data : /data/cache/interactions  feature_spec_file : feature_spec.yaml  epochs : 30  batch_size : 1048576  valid_batch_size : 1048576  factors : 64  layers : [256, 256, 128, 64]  negative_samples : 4  learning_rate : 0.0045  topk : 30  seed : None  threshold : 1.0  beta1 : 0.25  beta2 : 0.5  eps : 1e-08  dropout : 0.5  checkpoint_dir : /data/checkpoints/  load_checkpoint_path : None  mode : train  grads_accumulated : 1  amp : False  log_path : log.json  world_size : 1  distributed :

In [ ]:
!cd '{ncf_path}' && python3 -m torch.distributed.launch --nproc_per_node=1 \
    --use_env ncf.py -k 30 --data /data/cache/interactions \
    --mode pred --load_checkpoint_path /data/checkpoints/model.pth

In [105]:
import torch
from collections import defaultdict

# item_id, h3, chain_id, pred
a = torch.tensor(
   [[1, 3, 5, 1.0347e-01],
    [2, 3, 6, 8.4159e-03],
    [1, 3, 8, 2.5649e-01],
    [2, 3, 9, 2.5011e-03],
    [2, 3, 14, 7.0627e-01],
    [1, 3, 5, 2.9738e-02]])

u_idx = 0
h_idx = 1
i_idx = 2
p_idx = 3

u = 2
h = 3

a = a[a[:, u_idx].sort()[1]]
u_slice = a[a[:, u_idx] == u]
uh_slice = u_slice[u_slice[:, h_idx] == h]
scores_slice = uh_slice[:,i_idx:]
top = scores_slice[scores_slice[:, 1].topk(k=k)[1]]
mp = defaultdict(defaultdict)
mp[u][h] = top[:, 0]
print(mp[u][h])

tensor([[1.4000e+01, 7.0627e-01],
        [6.0000e+00, 8.4159e-03]])
tensor([14.,  6.])


tensor([1., 2.])